<a href="https://colab.research.google.com/github/jasminl/neuroblobs/blob/master/onnxsimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install onnx
# !pip install onnxruntime
# !pip install onnxruntime_extensions
# !pip install numpy

In [2]:
from onnx import helper, onnx_pb as onnx_proto, save_model
import onnxruntime as _ort
from onnxruntime_extensions import (
    onnx_op, PyCustomOpDef, make_onnx_model,
    get_library_path as _get_library_path)

nodes = [helper.make_node('CustomOpOne', ['input_1', 'input_2'], ['output_1'],
                              domain='ai.onnx.contrib'),
             helper.make_node('CustomOpTwo', ['output_1'], ['output_2'],
                              domain='ai.onnx.contrib'),
         helper.make_node('Add', ['output_2', 'output_1'], ['output'])]

@onnx_op(op_type="CustomOpOne", inputs=[PyCustomOpDef.dt_float, PyCustomOpDef.dt_float])
def custom_one_op(x, y):
    return np.add(x, y)

@onnx_op(op_type="CustomOpTwo",outputs=[PyCustomOpDef.dt_float])
def custom_two_op(f):
    return np.round(f)

input0 = helper.make_tensor_value_info(
    'input_1', onnx_proto.TensorProto.FLOAT, [3, 5])
input1 = helper.make_tensor_value_info(
    'input_2', onnx_proto.TensorProto.FLOAT, [3, 5])
output0 = helper.make_tensor_value_info(
    'output', onnx_proto.TensorProto.FLOAT, [3, 5])

graph = helper.make_graph(nodes, 'test0', [input0, input1], [output0])
model = helper.make_model(
    graph, opset_imports=[helper.make_operatorsetid('ai.onnx.contrib', 1)], ir_version=7)

save_model(model, 'mymodel.onnx')


In [3]:
from onnx import load_model
import numpy as np
so = _ort.SessionOptions()
so.register_custom_ops_library(_get_library_path())
new_model = load_model('mymodel.onnx')
#print(new_model)
sess = _ort.InferenceSession(new_model.SerializeToString(), so)

res = sess.run(None, {'input_1': np.random.rand(3, 5).astype(np.float32),
                      'input_2': np.random.rand(3, 5).astype(np.float32)})


In [4]:
print(new_model)

ir_version: 7
graph {
  node {
    input: "input_1"
    input: "input_2"
    output: "output_1"
    op_type: "CustomOpOne"
    domain: "ai.onnx.contrib"
  }
  node {
    input: "output_1"
    output: "output_2"
    op_type: "CustomOpTwo"
    domain: "ai.onnx.contrib"
  }
  node {
    input: "output_2"
    input: "output_1"
    output: "output"
    op_type: "Add"
  }
  name: "test0"
  input {
    name: "input_1"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 5
          }
        }
      }
    }
  }
  input {
    name: "input_2"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
          dim {
            dim_value: 5
          }
        }
      }
    }
  }
  output {
    name: "output"
    type {
      tensor_type {
        elem_type: 1
        shape {
          dim {
            dim_value: 3
          }
